In [1]:
using BenchmarkTools
using NPZ

Compile it with ifort, and check the exported symbols:
```
call "c:\Program Files (x86)\IntelSWTools\compilers_and_libraries_2019.5.281\windows\bin\ifortvars.bat" intel64
ifort -O2 /dll clipping.f90 -o ifort_clipping.dll
DUMPBIN /exports clipping.dll
```

If not on Windows, maybe add `-static-intel`, i.e.:
```
ifort -O2 /dll -static-intel clipping.f90 -o ifort_clipping.dll
```

Install MinGW for gfortran, make sure to link libgfortran statically or the DLL will not be accepted.
```
gfortran -O2 -shared -static-libgfortran clipping.f90 -o gfortran_clipping.dll
```

In [2]:
function ifort_area_of_intersection(a, b)
    ndim, nvertex, ntriangles = size(a)
    out = Vector{Float64}(undef, ntriangles)
    ccall(
        (:area_of_intersection, "./src/fortran/ifort_clipping.dll"),
        Nothing,
        (Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Float64}, Ptr{Float64}, Ptr{Float64}),
        Ref(ndim), Ref(nvertex), Ref(ntriangles), a, b, out,
    )
    return out
end

function gfortran_area_of_intersection(a, b)
    ndim, nvertex, ntriangles = size(a)
    out = Vector{Float64}(undef, ntriangles)
    ccall(
        (:area_of_intersection, "./src/fortran/gfortran_clipping.dll"),
        Nothing,
        (Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Float64}, Ptr{Float64}, Ptr{Float64}),
        Ref(ndim), Ref(nvertex), Ref(ntriangles), a, b, out,
    )
    return out
end

gfortran_area_of_intersection (generic function with 1 method)

In [3]:
a = permutedims(npzread("triangles_a.npy"), (3, 2, 1))
b = permutedims(npzread("triangles_b.npy"), (3, 2, 1))

2×3×1000000 Array{Float64,3}:
[:, :, 1] =
 0.13943   0.20051   0.0433536
 0.141259  0.103664  0.345891 

[:, :, 2] =
 0.638903  0.859061  0.286762
 0.725644  0.911425  0.768978

[:, :, 3] =
 0.127354  0.668901  0.166028
 0.614252  0.977322  0.912566

...

[:, :, 999998] =
 0.163299  0.918414  0.731335
 0.846918  0.664982  0.812965

[:, :, 999999] =
 0.581572  0.0160035  0.949833
 0.862413  0.445922   0.728067

[:, :, 1000000] =
 0.932028  0.104979  0.533061
 0.607966  0.153133  0.193289

In [4]:
@btime ifort_area_of_intersection(a, b)

  111.258 ms (2 allocations: 7.63 MiB)

1000000-element Array{Float64,1}:
 0.0                  
 0.0                  
 0.0015428417818308345
 0.0116674946363678   
 0.0                  
 0.012820796709396648 
 0.015439009483970728 
 0.0011013956528890018
 0.01195865718660706  
 0.01127765391313546  
 0.0                  
 0.009645649033342713 
 0.03162643130242185  
 ⋮                    
 0.00952080431982822  
 0.0010082222361460235
 0.0                  
 0.01320632733056133  
 0.03502262207396337  
 0.0                  
 0.0                  
 0.0                  
 0.021154967845624113 
 5.482193249288619e-7 
 0.019657919361185133 
 0.01220281881976658  

In [5]:
@btime gfortran_area_of_intersection(a, b)

  196.328 ms (2 allocations: 7.63 MiB)


1000000-element Array{Float64,1}:
 0.0                  
 0.0                  
 0.0015428417818308345
 0.0116674946363678   
 0.0                  
 0.012820796709396648 
 0.015439009483970728 
 0.0011013956528890018
 0.01195865718660706  
 0.01127765391313546  
 0.0                  
 0.009645649033342713 
 0.03162643130242185  
 ⋮                    
 0.00952080431982822  
 0.0010082222361460235
 0.0                  
 0.01320632733056133  
 0.03502262207396337  
 0.0                  
 0.0                  
 0.0                  
 0.021154967845624113 
 5.482193249288619e-7 
 0.019657919361185133 
 0.01220281881976658  